## SQL operations in PySpark using Python

With the advent of the [PySpark.sql API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html) and the DataFrame data-structure, the relationship between SQL and the PySpark api has become obvious. Where in the introduction notebook we looked at different ways of selecting columns, had a closer look at columns, but also investigated how to write good Python. In this notebook we will look at how to perform SQL operations using PySpark & Python.

Before we continue looking at PySpark Python equivalents to SQL, we should quickly answer two questions:

1. Can we use SQL instead of Python? 
2. Should we use SQL instead of Python?

The short answer are yes and no. To eleborate why should choose to use Python instead of SQL. In the introduction notebook I gave a few reasons why we should use Python, easy to learn, great ecosystem of libraries, PySpark being one of them, and flexible in use. I also mentioned the biggest drawback to Python; Python is prone to mistakes, or better it is easy to write faulty code in Python. I presented you with some steps to write better code:

1. Type your variables, functions, and classes.
2. Use good names for your variables, functions, and classes
3. Document your variables, functions, and classes using PyDoc and docstring
4. Test your methods, using DocTest (Ideal for interactive code) 

SQL as language is less powerful than Python is, you can do more with Python. For instance you can do graphics with Python, you cannot do that with SQL. Where Python is easy to read, SQL can be notoriously hard to read with subqueries, views, and/or multiple joins. SQL is more error prone than Python. The most important reason why you should use Python instead of SQL; Python is very good at breaking up problems into smaller easier to solve problems, which you can than combine in to a solution, SQL is not. The one thing going for SQL is that it is in general faster than Python. However, we can speed up Python to match SQL if necessary.  

As with the previous notebook, this notebook continuous with PySpark Python equivalents to SQL. That means looking at how to perform joins, order, group things, and using aggregate functions. 

We start with the same broadcast logs as the base DataFrame. ALL CSV files can be found at: [GitHub](https://github.com/jonesberg/DataAnalysisWithPythonAndPySpark-Data/tree/trunk/broadcast_logs)

You have to remember to change the path if you run this code.

In [ ]:
# cell for imports

import doctest
import os

import numpy as np
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.errors import AnalysisException
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.session import SparkSession

Creating a session 

In [ ]:
spark: SparkSession = SparkSession.builder.appName("Python Joins").getOrCreate()
spark.sparkContext.setLogLevel("Error")

As the first part of this notebook is first and foremost about joins we will need two DataFrames.

In [ ]:
path_one: str = "./Downloads"
broadcast_logs: DataFrame = spark.read.csv(
    path=os.path.join(path_one, "BroadcastLogs_2018_Q3_M8.CSV"),
    sep="|",
    header=True,
    inferSchema=True,
    timestampFormat="yyyy-MM-dd",
)

path_two = "./ProgrammingProjects/SparkTest/DataAnalysisWithPythonAndPySpark-Data-trunk/broadcast_logs/ReferenceTables/"

log_identifier: DataFrame = spark.read.csv(
    path=os.path.join(path_two, "LogIdentifier.csv"),
    sep="|",
    header=True,
    inferSchema=True,
)

A quick inspection of `log_identifier`, we have used `broadcast_logs` before.

In [ ]:
log_identifier.printSchema()

In [ ]:
log_identifier.show(n=5, truncate=False)

## The different joins in PySpark

The purpose of a join is to answer one of two questions: what happens when the return value of the predicate is true, and conversely.

#### The types of joins

1. The inner join returns the record if the predicate is true; otherwise, it drops it. Standard join option.
2. The left/right inner join will join the unmatched records from the left/right, filling the columns from the right/left with NULL.
3. The full outer join: the fusion from the left and right inner join will add the unmatched records from the left and right padding with NULL.
4. The left semi join: same as an inner join, it keeps the columns on the left but will discard those rows that fulfil the predicate with more than one record from the right.
5. The left anti-join keeps only those records from the left that do not match with the predicate; it will drop all those that do.
6. A Cartesian product, or cross-join. Pyspark has a specific `crossJoin()` method. Obviously, using this method will explode your DataFrame into enormous proportions!

All these different joins are parameter values in the `join` method, with the key word "how" as in how=inner. Inner is the standard option in the `join` method.

Performance-wise, you will need to have both tables on the same computer; if not, PySpark will perform the join on the network, which will quickly cost you >99% of performance.


In [ ]:
logs_and_channels_verbose: DataFrame = broadcast_logs.join(
    other=log_identifier,
    on=broadcast_logs["LogServiceID"] == log_identifier["LogServiceID"],
    how="inner",
)

A join performed like this will work, but will lead to ambiguity, see the following error.

In [ ]:
try:
    logs_and_channels_verbose.select("LogServiceID")
except AnalysisException as error:
    print(error)

If you print the schema you will see the two LogServiceIDs

In [ ]:
logs_and_channels_verbose.printSchema()

We can make the join without the `==` and drop the offending column in one swoop

In [ ]:
logs_and_channels = broadcast_logs.join(
    other=log_identifier, on="LogServiceID", how="inner"
).drop(log_identifier["LogServiceID"])

Now we have only one  column "LogServiceID"

In [ ]:
len([col for col in logs_and_channels.columns if col == "LogServiceID"])

We can join several tables in one go.

In [ ]:
path: str = (
    "./ProgrammingProjects/SparkTest/DataAnalysisWithPythonAndPySpark-Data-trunk/broadcast_logs/"
)

cd_category: DataFrame = spark.read.csv(
    path=os.path.join(path, "ReferenceTables/CD_Category.csv"),
    sep="|",
    header=True,
    inferSchema=True,
).select(
    "CategoryID",
    "CategoryCD",
    F.col("EnglishDescription").alias("Category_Description"),
)

cd_program_class: DataFrame = spark.read.csv(
    path=os.path.join(path, "ReferenceTables/CD_ProgramClass.csv"),
    sep="|",
    header=True,
    inferSchema=True,
).select(
    "ProgramClassID",
    "ProgramClassCD",
    F.col("EnglishDescription").alias("ProgramClass_Description"),
)
# joining 3 tables together
full_log: DataFrame = logs_and_channels.join(
    other=cd_category, on="CategoryID", how="left"
).join(other=cd_program_class, on="ProgramClassID", how="left")

In [ ]:
full_log.printSchema()

In [ ]:
full_log.select("ProgramClassCD", "ProgramClass_Description", "Duration").show(
    n=5, truncate=False
)

The duration is very unclear; we would want to use the same base, seconds in this case.
Because we are using `F.col` we can use the [`substr`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.substr.html#pyspark.sql.Column.substr) method from the [column API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/column.html) (one of PySpark core classes), we can transform duration.

In [ ]:
full_log: DataFrame = full_log.withColumn(
    "duration_seconds",
    (
        F.col("Duration").substr(1, 2).cast("int") * 3600
        + F.col("Duration").substr(4, 2).cast("int") * 60
        + F.col("Duration").substr(7, 2).cast("int")
    ),
)

In [ ]:
full_log.select("ProgramClassCD", "ProgramClass_Description", "duration_seconds").show(
    n=5, truncate=False
)

#### Grouping, Ordering, and Having 
The `groupBy` and `orderBy` functions do pretty much what you would expect from knowing SQL. However, there is a major difference in PySpark: the return value:

- `DataFrame.groupBy → DataFrameGroupBy`
- `DataFrame.orderBy → pyspark.sql.dataframe.DataFrame`

We would need to perform an operation on the DataFrameGroupBy to get a DataFrame. Also, you will see both `groupby` and `groupBy` but you won't see `orderby` just `orderBy`. To avoid confusing myself, I just use the capitalised B for both methods. 

As having is just filtering after grouping there is no Python equivalent, just use `filter` or `where`, the latter being alias for the former.

In [ ]:
full_log.groupBy("ProgramClassCD", "ProgramClass_Description")

## aggregating

The `agg` method totals on a given axis and returns a DataFrame, we can use it on GroupedData. The `agg` method takes an aggregation function. Usually one of the built-in aggregation functions, such as avg, max, min, sum, and count.

The [agg](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.GroupedData.agg.html#pyspark.sql.GroupedData.agg) method is an important method; I advise you to look at the documentation. 

The `agg` is also a method of the DataFrame class that aggregates on the entire DataFrame without grouping. Instead of using a `F.sum(column)`, we could have added a dictionary to `agg`; with the column we want to aggregate over and the function we want to use: `agg({'duration_seconds':'sum'})`. However, that would prevent us from aliasing the column, and we would have to rename it later.

In [ ]:
analysis_one: DataFrame = (
    full_log.groupBy("ProgramClassCD", "ProgramClass_Description")
    .agg(F.sum("duration_seconds").alias("duration_total"))
    .orderBy("duration_total", ascending=False)
)
analysis_one.show(n=20, truncate=False)

You now have another DataFrame you could use for analysis or report with

In [ ]:
analysis_one.select("duration_total").summary().show()

## Mixing Python with SQL


Spark knows two SQL dialects: the ANSI standard SQL and HiveQL, Hive is an open-source data warehouse. This notebook will use ANSI SQL and some built-in SQL functions that PySpark provides. Databricks provides you with the warehouse, so there is no need for Hive.

**Declarative programming with SQL**

SQL is a declarative language; programmes describe their desired results without explicitly listing commands or steps that must be performed. The DML (data manipulation language) part of SQL basically splits the programmes into two parts:

- Operations, comprising select columns from a target and functions such as count, max, but also aliasing; `select whatever as alias, count(*) from target`
- Conditions: comprising  conditions where or having, grouping, ordering, and filtering; `where condition1 and condition 2, group by condition1 having something order by`

**Imperative programming with Python**

Python is an imperative programming language. In short, some object represents the state of a computer program, and the program gives instructions on how to change that state. The standard state in PySpark is the DataFrame, PySpark chains the transformations and actions on that DataFrame.

SQL is faster than Python. SQL is native to Spark, and the data Spark uses is almost always tabular. SQL, which is an application of relational algebra, is a domain-specific language; SQL is made for relational data. Whereas Python can handle relational data but is not made for it, which is why Python programmers usually use an ORM like SQLAlchemy.

Why use Python at all? Like all imperative languages, Python can easily breakdown large problems into small problems, which can be built up in an algorithmic manner to solve your larger problems. 

I will start this notebook with a bit of Python. As I said, we first need to get a state we can change. In PySpark, the state is a DataFrame.

In [ ]:
path: str = (
    "./ProgrammingProjects/SparkTest/DataAnalysisWithPythonAndPySpark-Data-trunk/"
)

elements: DataFrame = spark.read.csv(
    path=path + "elements/Periodic_Table_Of_Elements.csv", header=True, inferSchema=True
)
elements.printSchema()

In [ ]:
elements.select("AtomicNumber", "Element", "MeltingPoint", "BoilingPoint").show(n=6)

In [ ]:
elements.filter(F.col("AtomicNumber") == 79).select(
    "AtomicNumber", "Element", "MeltingPoint", "BoilingPoint"
).show()

Say we wanted to get following query in SQL:   

```
SELECT
  element,
  period,
  count(*)
FROM elements
WHERE phase='gas'
GROUP BY period;
```

We could write the following line of code in Python

In [ ]:
elements.where(F.col("Phase") == "gas").groupBy("period").count().show(n=5)

The PySpark `groupBy` automatically selects the column. A short cut, if
you only want a few columns.  

We could use the SQL too, despite having a DataFrame as datastructure. We need to create a `view`:

In [ ]:
elements.createOrReplaceTempView(name="elements")

spark.sql(
    """
SELECT
  period,
  count(*)
FROM elements
WHERE phase='gas'
GROUP BY period;
"""
).show(n=5)

You might have noticed we have not assigned a name, so where does PySpark store this view? 
Within the session is the short answer. We can inspect the sessions' [catalog](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Catalog.html). 

In [ ]:
spark.catalog.listTables()

#### View vs. Table

A quick side note on the difference between a view and a table. The important difference is that a table is an existing entity in the database or data warehouse, whereas a view is the instructions on how to create the table. A view is therefor also known as a virtual table.

You will notice this difference in use: a table can be placed directly in memory (if the memory available is big enough), and a view needs to be computed every single time a session is started. This has an immediate effect on performance. Operations on a table are faster.

Why use a view? If you want to create a table from different tables but intend to use it sparsely, it wouldn't be worth creating an actual object in persistent storage for it. Then you should use a view.

PySpark and views are slightly different; you have already created the object (the DataFrame you are referencing) when making it a view. Performance-wise, this would be on par with loading a table in memory.

A view created with `createOrReplaceTempView` only exists as long as the DataFrame exist, that is, for the duration of the session.

A view created with `createOrReplaceGlobalView` exists as long as there is a Spark application. This is really only necessary if you have an application that requires multiple SparkSessions to cooperate.

In [ ]:
q1: DataFrame = spark.read.csv(path="./drive_stats_Q1", header=True, inferSchema=True)
q2: DataFrame = spark.read.csv(path="./drive_stats_Q2", header=True, inferSchema=True)
q3: DataFrame = spark.read.csv(path="./drive_stats_Q3", header=True, inferSchema=True)
q4: DataFrame = spark.read.csv(path="./drive_stats_Q4", header=True, inferSchema=True)

## Backblaze
Backblaze is a cloud storage provider that, among other things, offers a service for IaaS. For instance, for storage in the cloud. Backblaze offers you many hard disk drive options; how would you know which one to choose? You want the best fitting capacity with the lowest failure rate. PySpark can determine this.

For the following programming examples, you will need to download the quarterly files for 2019 (q1–q4) from [Backblaze](https://www.backblaze.com/cloud-storage/resources/hard-drive-test-data).

You need to unpack all the folders; PySpark has no automatic unzipping. Furthermore, you need to use your own paths if you run the code in the notebook.

I happen to know that Q4 has more columns than the other quarters.  This information you won't get from calling `printSchema`,  `show`, or `count`. But we can revert to simple Python if we want to know the number of columns.

In [ ]:
len(q3.columns) - len(q4.columns)

#### Python helpers
Now we could imagine wanting to create a single DataFrame from multiple CSV files quite often. This is where Python comes in. The advantage of using Python over SQL is that Python is a much more powerful language than SQL. In technical terms, Python is Turing complete; SQL is not. For instance, instead of manually having to ask and compare the size of the columns in a table or DataFrame, we can write helper functions that we can use over and over again. 

Normally, I would put these helper functions in a separate module or even a class (the CSV Fusion class?) and not in the script. If you import the module with your imports, you will always have access to a host of useful helper functions. Save yourself the hassle of repeatedly having to write the same or similar code.

As per the 4 steps these helpers should be typed, have names that convey their use, they should be documented, and they should be tested.

In [ ]:
def same_column_size(dfs: list[DataFrame]) -> bool:
    """
    desc: Function to compare column size
    >>> same_column_size([q1,q2])
    True
    >>> same_column_size([q1,q4])
    False
    """
    size = len(dfs[0].columns)
    for df in dfs:
        if len(df.columns) != size:
            return False
    return True


def compare_column_size(dfs: list[DataFrame]) -> list[int]:
    """
    desc: Function that returns the column sizes
    >>> compare_column_size([q1,q2,q3,q4])
    [129, 129, 129, 131]
    """
    result = []
    for i in range(len(dfs)):
        result.append(len(dfs[i].columns))
    return result


doctest.testmod()

Now that we know that one quarter has more columns, we should either drop those columns 
or add them to the other quarters. We are going to do the latter. Again, this is something that you could consider doing in a separate module or in a class. At the end of this notebook, I will show you a different way to do the same.

In [ ]:
q4_columns_extra: set[str] = set(q4.columns) - set(q1.columns)

# add the columns, fill them with None value
for col in q4_columns_extra:
    q1 = q1.withColumn(col, F.lit(None).cast(T.StringType()))
    q2 = q2.withColumn(col, F.lit(None).cast(T.StringType()))
    q3 = q3.withColumn(col, F.lit(None).cast(T.StringType()))

# creating the full set of data (equivalent to an SQL union all)
back_blaze_2019: DataFrame = (
    q1.select(q4.columns)
    .union(q2.select(q4.columns))
    .union(q3.select(q4.columns))
    .union(q4)
)

# setting the layout for each column according to the schema
back_blaze_2019: DataFrame = back_blaze_2019.select(
    [
        F.col(x).cast(T.LongType()) if x.startswith("smart") else F.col(x)
        for x in back_blaze_2019.columns
    ]
)

In [ ]:
# Now create a view for the SQL
back_blaze_2019.createOrReplaceTempView("backblaze_2019_view")
spark.catalog.listTables()

In [ ]:
spark.sql(
    """
    SELECT model, serial_number 
    FROM backblaze_2019_view 
    where failure = 1
    """
).show(n=5, truncate=False)

In [ ]:
# in Python
back_blaze_2019.where("failure=1").select("model", "serial_number").show(
    n=5, truncate=False
)

Placeholders can create unclear code if used thoughtless. Compare the queries and determine which is 
clearer from the point of view of the reader.

In [ ]:
spark.sql(
    """
    SELECT model, min(capacity_bytes / pow(1024, 3)) as min_GB, max(capacity_bytes / pow(1024, 3)) as max_GB
    FROM backblaze_2019_view
    GROUP BY 1
    ORDER BY 3 DESC
    """
).show(n=5)

In [ ]:
spark.sql(
    """
    SELECT model, min(capacity_bytes / pow(1024, 3)) as min_GB, max(capacity_bytes / pow(1024, 3)) as max_GB
    FROM backblaze_2019_view
    GROUP BY model
    ORDER BY max_GB DESC
    """
).show(n=5)

In [ ]:
# in Python
back_blaze_2019.groupBy(F.col("model")).agg(
    F.min(F.col("capacity_bytes") / F.pow(1024, 3)).alias("min_GB"),
    F.max(F.col("capacity_bytes") / F.pow(1024, 3)).alias("max_GB"),
).orderBy(F.col("max_GB"), ascending=False).show(n=5)

#### The having clause
What if you want to filter after grouping? In SQL, you have the having clause. In Python, you just use the `filter` function or its alias `where` after the grouping. Just remember that `groupBy` returns a grouping object. You need a DataFrame to use `filter`. In general, this won't be a problem, as you will want to use some aggregation over the group. 

In [ ]:
spark.sql(
    """
    SELECT model, min(capacity_bytes / pow(1024, 3)) as min_GB, max(capacity_bytes / pow(1024, 3)) as max_GB
    FROM backblaze_2019_view
    GROUP BY model
    HAVING min_GB != max_GB
    ORDER BY max_GB DESC
    """
).show(n=5)

In [ ]:
# equivalent Python where first group then filter
back_blaze_2019.groupBy(F.col("model")).agg(
    F.min(F.col("capacity_bytes") / F.pow(1024, 3)).alias("min_GB"),
    F.max(F.col("capacity_bytes") / F.pow(1024, 3)).alias("max_GB"),
).filter(F.col("min_GB") != F.col("max_GB")).orderBy(
    F.col("max_GB"), ascending=False
).show(
    n=5
)

#### data definition language (DDL)

Sofar we have focussed on using the DML part of SQL. We can equally use DDL in PySpark. Let me first create a new view.

In [ ]:
back_blaze_2019.createOrReplaceTempView("drive_stats_view")
spark.catalog.listTables()

In [ ]:
spark.sql(
    """
    CREATE OR REPLACE TEMP VIEW drive_days AS
    SELECT model, count(*)  AS drive_days
    FROM drive_stats_view
    GROUP BY model
    """
)

spark.sql(
    """
    CREATE OR REPLACE TEMP VIEW failures AS
    SELECT model, count(*) AS failures
    FROM drive_stats_view
    WHERE failure = 1
    GROUP BY model
    """
)

In [ ]:
spark.catalog.listTables()

We have added two views to our catalog.

Back to the difference between a view and a table: A table is an actual object in persistent memory. A Python object, for instance, a DataFrame, is not stored in persistent memory. It exists very much as a set of instructions (the class DataFrame) on how to make something and only goes into memory once compiled. A view is thus much more equal to a Python object than a table is. 

Of course, we know how to do very much the same in Python as we just did in SQL.

In [ ]:
drive_days: DataFrame = back_blaze_2019.groupBy(F.col("model")).agg(
    F.count(F.col("*")).alias("drive_days")
)
drive_days.show(n=5)

In [ ]:
failures: DataFrame = (
    back_blaze_2019.filter("failure == 1")
    .groupBy("model")
    .agg(F.count(F.col("*")).alias("failures"))
)
failures.show(n=5)

## Relational Operators
Python knows relational operators, such as `union` and `intersect_update`. As a table is a set of rows, we can use the set relational operators in Spark too. Given sets **A** and **B**, these operators are:
1. Union. $A \cup B = \ x \in A \lor x \in B$. In practice, you will probably mostly see the **union**.
2. Intersect. $A \cap B = x \in A \land x \in B$  
3. Except. $A / B =  x \in A, x\ni B$

It is important that when you use a set operator on two tables, they have the same number of columns and that those columns have comparable data types! 

Also, you should be aware that there is a difference between a **SQL union**, which removes duplicates, and a **PySpark union**, which does not do so. There is a good reason why the PySpark union does not remove duplicates; this is a very computationally expensive operation in a distributed environment. This is why you should not use plain union in SQL but the **union all** in distributed environments.

Why do you need this? Simply because you quite often want to join the data from multiple tables in a bigger table. We have done this above using Python, i.e., `.union(q2.select(q4.columns))` Now we will do so in SQL.

In [ ]:
# create a string for use in sql
columns_backblaze = ", ".join(q4.columns)

# create the views
q1.createOrReplaceTempView("Q1")
q2.createOrReplaceTempView("Q2")
q3.createOrReplaceTempView("Q3")
q4.createOrReplaceTempView("Q4")

# create the same view as the backblaze_2019 DataFrame
spark.sql(
    """
    CREATE OR REPLACE TEMP VIEW backblaze_2019 AS
    SELECT {col} from Q1 UNION ALL
    SELECT {col} from Q2 UNION ALL
    SELECT {col} from Q3 UNION ALL
    SELECT {col} from Q4
    """.format(
        col=columns_backblaze
    )
)
spark.catalog.listTables()

#### Union vs. Join
In my mind, there always seems to be a bit of confusion between a **join** and a **union**. A union basically adds rows to a column, and a join adds columns to tables; it makes a cartesian product and subsets it, to be exact.

Of course, we can use join:

In [ ]:
spark.sql(
    """
    SELECT drive_days.model, drive_days, failures
    FROM drive_days
    LEFT JOIN failures
    ON drive_days.model = failures.model
    """
).show(n=5, truncate=False)

In Python, we can achieve the same, but in my opinion, cleaner.

In [ ]:
drive_days.join(other=failures, on="model", how="left").show(n=5)

## Complex queries
Now we come to why I think that Python is the better choice for PySpark. Though SQL is enormously powerful for handling tabular data, it is very hard to structure SQL. SQL tends to become quickly difficult to read, especially when you start using subqueries and subqueries in subqueries.

Consider the following somewhat complex SQL query:

In [ ]:
spark.sql(
    """
    SELECT failures.model, failures / drive_days AS failure_rate
    FROM (
        SELECT model, count(*) AS drive_days
        FROM drive_stats_view
        GROUP BY model
        ) drive_days
    INNER JOIN (
        SELECT model, count(*) AS failures
        FROM drive_stats_view
        WHERE failure = 1
        GROUP BY model
        ) failures
    ON drive_days.model = failures.model
    ORDER BY failure_rate DESC
    """
).show(n=5)

In Python, we can breakdown this problem into three separate steps:

1. Create a drivedays DataFrame
2. Create a failures DataFrame
3. Create a failure_rates DataFrame by joining the first two.
   
I have created the first two already, but let's copy them in for clarity.

In [ ]:
# step 1
drive_days: DataFrame = back_blaze_2019.groupBy(F.col("model")).agg(
    F.count(F.col("*")).alias("drive_days")
)
# step 2
failures: DataFrame = (
    back_blaze_2019.filter("failure == 1")
    .groupBy("model")
    .agg(F.count(F.col("*")).alias("failures"))
)
# step 3
failure_rates: DataFrame = (
    drive_days.join(other=failures, on="model", how="inner")
    .withColumn(
        colName="failure_rate", col=F.round(F.col("failures") / F.col("drive_days"), 5)
    )
    .orderBy(F.col("failure_rate").desc())
)

In [ ]:
failure_rates.show(n=5, truncate=False)

Of course, SQL has common table expressions that you can use to break down complex code. However,
I feel that CTEs do not really make code less complex. See the example using CTE:

In [ ]:
spark.sql(
    """
    WITH drive_days as (
        SELECT model, count(*) AS drive_days
        FROM drive_stats_view
        GROUP BY model
        ),
         failures as (
        SELECT model, count(*) AS failures
        FROM drive_stats_view
        WHERE failure = 1
        GROUP BY model
        )
    SELECT failures.model, failures / drive_days AS failure_rate
    FROM drive_days
    INNER JOIN failures
    ON drive_days.model = failures.model
    ORDER BY failure_rate DESC
    """
).show(n=5)

#### Namespaces
There is a major disadvantage to the Python solution I presented. Our intermediary objects (drive_days and failures) are still available. This is not what we want; they were mere steps in our solution. In Python technical terms, they are in the general [namespace](https://github.com/lausandt/Programming-in-Python-notebooks/blob/master/StructuredProgramming.ipynb). A namespace is a mapping of names to objects; in Python, these namespaces take the form of a dictionary. To see all the objects in the namespace, just call `dir`

In [ ]:
dir()

#### Secure Python solution
As you can see, our intermediary results are in the general namespace of the script. Available for all that have access to change and, with that, to change our answer. What we should do is encapsulate the intermediary results in their own namespace. How? Simple: we encapsulate the calculations with a function, narrowing the namespace of the dataframes to the function.

This function is too complicated to test with DoctTest and would require some mocking of test data for the test. Complicated functions like this should be written in a separate script and tested with a test framework like [Pytest](https://docs.pytest.org/en/8.2.x/).

In [ ]:
def failure_rate(drive_stats: DataFrame) -> DataFrame:
    """
    desc:Function to show the failure rate of Hard Disk Drives
    """
    drive_daysA = drive_stats.groupBy(F.col("model")).agg(
        F.count(F.col("*")).alias("drive_days")
    )
    failuresA = (
        drive_stats.filter(F.col("failure") == 1)
        .groupBy(F.col("model"))
        .agg(F.count(F.col("*")).alias("failures"))
    )
    answerA = (
        drive_daysA.join(other=failuresA, on="model", how="inner")
        .withColumn("failure_rate", F.round(F.col("failures") / F.col("drive_days"), 5))
        .orderBy(F.col("failure_rate").desc())
    )
    return answerA

In [ ]:
fails: DataFrame = failure_rate(drive_stats=back_blaze_2019)
fails.show(n=5)

You can check there are no sub results in the general namespace

In [ ]:
"drive_daysA" in dir() or "failuresA" in dir() or "answerA" in dir()

#### Conclusion
This is a long notebook and there is more to say about PySpark, SQL, and Python. Those are the more advanced topics, such as:

1. Using functional style programming
2. SQL Expressions in PySpark
3. Caching
4. Windows functions